# 005 TEST

# Function Call

In [1]:
%pip install ipympl

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install tiktoken openai pandas matplotlib plotly scikit-learn numpy python-dotenv matplotlib widget

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement widget (from versions: none)
ERROR: No matching distribution found for widget
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()  


import openai
client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Function Call

In [4]:
%pip install scipy tenacity tiktoken termcolor openai requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import json
import requests
import os
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo"

/Users/prigogine/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + os.getenv("OPENAI_API_KEY"),
    }

    json_data = {"model": model, "messages": messages}

    if functions is not None:
        json_data.update({"functions": functions})

    if function_call is not None:
        json_data.update({"function_call": function_call})

    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response

    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [7]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))

        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))

        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant[function_call]: {message['function_call']}\n", role_to_color[message["role"]]))

        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant[content]: {message['content']}\n", role_to_color[message["role"]]))

        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [8]:
functions = [
    {
        "name": "get_current_weather",  
        "description": "Get the current weather",  
        "parameters": {  
            "type": "object",
            "properties": {  
                "location": {  
                    "type": "string",  
                    "description": "The city and state, e.g. San Francisco, CA",  
                },
                "format": {  
                    "type": "string",  
                    "enum": ["celsius", "fahrenheit"],  
                    "description": "The temperature unit to use. Infer this from the users location.",  
                },
            },
            "required": ["location", "format"],  
        },
    },
    
    {
        "name": "get_n_day_weather_forecast",  
        "description": "Get an N-day weather forecast",  
        "parameters": {  
            "type": "object",
            "properties": {  
                "location": {  
                    "type": "string",  
                    "description": "The city and state, e.g. San Francisco, CA",  
                },
                "format": {  
                    "type": "string",  
                    "enum": ["celsius", "fahrenheit"],  
                    "description": "The temperature unit to use. Infer this from the users location.",  
                },
                "num_days": {  
                    "type": "integer",  
                    "description": "The number of days to forecast",  
                }
            },
            "required": ["location", "format", "num_days"]  
        },
    },
]


In [9]:
messages = []
messages.append({
    "role": "system",  
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  
})

messages.append({
    "role": "user",  
    "content": "What's the weather like today"  
})

chat_response = chat_completion_request(
    messages, functions=functions
)

assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant[content]: Sure, may I know your current location so that I can provide you with the weather information?



In [10]:
messages.append({
    "role": "user",  
    "content": "I'm in Piran City."  
})

chat_response = chat_completion_request(
    messages, functions=functions
)

assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant[content]: Sure, may I know your current location so that I can provide you with the weather information?

user: I'm in Piran City.

assistant[function_call]: {'name': 'get_current_weather', 'arguments': '{"location":"Piran City","format":"celsius"}'}



In [11]:
messages = []
messages.append({
    "role": "system",  
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

messages.append({
    "role": "user",  
    "content": "what is the weather going to be like in Piran city over the next x days"
})

chat_response = chat_completion_request(
    messages, functions=functions
)

assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Piran city over the next x days

assistant[content]: Sure, I can help you with that. How many days would you like to forecast the weather for in Piran city?



In [12]:
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "5 days"
})

chat_response = chat_completion_request(
    messages, functions=functions
)

assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Piran city over the next x days

assistant[content]: Sure, I can help you with that. How many days would you like to forecast the weather for in Piran city?

user: 5 days

assistant[function_call]: {'name': 'get_n_day_weather_forecast', 'arguments': '{"location":"Piran","format":"celsius","num_days":5}'}



In [13]:
import sqlite3
conn = sqlite3.connect("data/chinook.db")
print("opened db successfully")

opened db successfully


In [14]:
def get_table_names(conn):
    table_names = []  
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names  


def get_column_names(conn, table_name):
    column_names = []  
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names  


def get_database_info(conn):
    table_dicts = []  
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)  
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts  

In [15]:
database_schema_dict = get_database_info(conn)

database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [16]:
database_schema_dict

[{'table_name': 'Album', 'column_names': ['AlbumId', 'Title', 'ArtistId']},
 {'table_name': 'Artist', 'column_names': ['ArtistId', 'Name']},
 {'table_name': 'Customer',
  'column_names': ['CustomerId',
   'FirstName',
   'LastName',
   'Company',
   'Address',
   'City',
   'State',
   'Country',
   'PostalCode',
   'Phone',
   'Fax',
   'Email',
   'SupportRepId']},
 {'table_name': 'Employee',
  'column_names': ['EmployeeId',
   'LastName',
   'FirstName',
   'Title',
   'ReportsTo',
   'BirthDate',
   'HireDate',
   'Address',
   'City',
   'State',
   'Country',
   'PostalCode',
   'Phone',
   'Fax',
   'Email']},
 {'table_name': 'Genre', 'column_names': ['GenreId', 'Name']},
 {'table_name': 'Invoice',
  'column_names': ['InvoiceId',
   'CustomerId',
   'InvoiceDate',
   'BillingAddress',
   'BillingCity',
   'BillingState',
   'BillingCountry',
   'BillingPostalCode',
   'Total']},
 {'table_name': 'InvoiceLine',
  'column_names': ['InvoiceLineId',
   'InvoiceId',
   'TrackId',
   '

In [17]:
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Output should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    }
]


In [18]:
def ask_database(conn, query):
    try:
        results = str(conn.execute(query).fetchall())  
    except Exception as e:  
        results = f"query failed with error: {e}"
    return results  

def execute_function_call(message):    
    if message["function_call"]["name"] == "ask_database":
        query = json.loads(message["function_call"]["arguments"])["query"]
        results = ask_database(conn, query)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results  

In [19]:

messages = []
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})
# messages.append({"role": "user", "content": "Hi, who are the top 5 artists by number of tracks?"})
# chat_response = chat_completion_request(messages, functions)

# assistant_message = chat_response.json()["choices"][0]["message"]
# messages.append(assistant_message)

# if assistant_message.get("function_call"):
#     results = execute_function_call(assistant_message)
#     messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})

# pretty_print_conversation(messages)

In [20]:
messages.append({"role": "user", "content": "What is the name of the album with the most tracks?"})
chat_response = chat_completion_request(messages, functions)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)

if assistant_message.get("function_call"):
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})

pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: What is the name of the album with the most tracks?

assistant[function_call]: {'name': 'ask_database', 'arguments': '{"query":"SELECT Album.Title, COUNT(Track.TrackId) AS TrackCount\\nFROM Album\\nJOIN Track ON Album.AlbumId = Track.AlbumId\\nGROUP BY Album.Title\\nORDER BY TrackCount DESC\\nLIMIT 1;"}'}

function (ask_database): [('Greatest Hits', 57)]

